In [1]:
#! pip install cherrypy

In [2]:
#! pip install markupsafe==2.0.1

In [3]:
#! pip install itsdangerous==2.0.1

In [4]:
#! pip install -U Werkzeug==0.16.0

In [5]:
import pandas as pd
import numpy as np
from app_function import view_classifica, view_player_page, view_role_explorer, view_team_builder, view_builded_team
from utils_function import slot_handler

In [6]:
player_info = pd.read_csv('../dataset/stat_info^(Stagione 2021_22).csv',index_col=[0])
player_hist = pd.read_csv('../dataset/player_history^(Stagione 2021_22).csv',index_col=[0])

player_hist.Voto = player_hist.Voto.astype(float)
player_hist.FV = player_hist.FV.astype(float)
player_hist.Bonus = player_hist.Bonus.astype(float)
player_hist.Malus = player_hist.Malus.astype(float)

all_names = player_info.Nome.unique().tolist()
all_teams = player_info.Squadra.unique().tolist()

classifica = pd.read_csv('../dataset/final_classifica^(Stagione 2021_22).csv',index_col=[0])

In [ ]:
from flask import Flask, render_template,request, redirect, url_for, session

app = Flask(__name__)

@app.route("/")
def home_page():
    session['actual_nome_selezionato'] = all_names[0]
    session['actual_team_list'] = []
    session['actual_modulo_scelto'] = '3-4-3'
    session['actual_role_selected'] = 'P'
    session['actual_min_pres'] = 0
    session['actual_team_selected'] = 'Tutte'
    session['actual_slot_asta'] = 'Tutti'
    session['actual_show_fav'] = True
    session['actual_show_not_fav'] = True
    session['actual_sort_by'] = [None,None]

    return render_template('home_page.html')

@app.route("/player-page")
def player_page():
    return view_player_page(player_info,player_hist,all_names,nome  = session['actual_nome_selezionato'])
    
@app.route('/player-page', methods=['POST'])
def handle_name():
    nome_selected = request.form.get('nome_selezionato')
    session['actual_nome_selezionato'] = nome_selected
    return view_player_page(player_info,player_hist,all_names,session['actual_nome_selezionato'])

@app.route('/classifica', methods=("POST", "GET"))
def classfica_page():
    return view_classifica(classifica)

@app.route('/role-explorer')
def role_page():
    fav_file = open("../tmp/favourite.txt", "r")
    return view_role_explorer(player_info, all_teams, fav_file = fav_file )

@app.route('/role-explorer', methods=['POST'])
def handle_role_explorer():
    
    if 'sort' in request.form:
        sort_by = request.form.getlist('sort')
        if session['actual_sort_by'][0] == sort_by[0]:
            session['actual_sort_by'] = [sort_by[0],False]
        else:
            session['actual_sort_by'] = [sort_by[0],True]
    
    if 'checked_fav' in request.form or  'checked_not_fav' in request.form:
        
        if 'checked_fav' in request.form:
            show_fav = True
        else:
            show_fav = False

        if 'checked_not_fav' in request.form:
            show_not_fav = True
        else:
            show_not_fav = False

        session['actual_show_fav'] = show_fav
        session['actual_show_not_fav'] = show_not_fav
        
    if 'slot_selector' in request.form:
        name_and_slot_list = request.form.getlist('slot_selector')
        slot_handler(name_and_slot_list)

    if 'add_to_fav' in request.form:
        
        name_for_fav = request.form.get('add_to_fav')    
        file_in = open('../tmp/favourite.txt',"r")

        names_favourite = file_in.read().splitlines()
        if '' in names_favourite: 
            names_favourite.remove('')
        file_in.close()
        
        if name_for_fav in names_favourite:
            file_write = open('../tmp/favourite.txt',"w")
            names_favourite.remove(name_for_fav)
            for names in names_favourite:
                if names != '':
                    file_write.write(names + '\n')
            file_write.close()
        else:
            file_append = open('../tmp/favourite.txt',"a")
            file_append.write(name_for_fav +'\n')
            file_append.close()
        
    if 'min_pres' in request.form:
        min_pres = request.form.get('min_pres')
        session['actual_min_pres'] = min_pres
        
    if 'role_selected' in request.form:
        min_pres = request.form.get('role_selected')
        session['actual_role_selected'] = min_pres

    if 'team_selected' in request.form:
        min_pres = request.form.get('team_selected')
        session['actual_team_selected'] = min_pres
    
    if 'slot_asta' in request.form:
        slot_asta = request.form.get('slot_asta')
        session['actual_slot_asta'] = slot_asta

    fav_file = open("../tmp/favourite.txt", "r")
    
    return view_role_explorer(player_info, all_teams, fav_file = fav_file,
                              slot = session['actual_slot_asta'],
                              role = session['actual_role_selected'], 
                              filter_presence = session['actual_min_pres'], 
                              team_selected = session['actual_team_selected'],
                              show_fav = session['actual_show_fav'],
                              show_not_fav = session['actual_show_not_fav'],
                              sort_by = session['actual_sort_by'])

@app.route('/team-builder')
def team_builder():
    return view_team_builder(player_info, session['actual_modulo_scelto'] )
            
@app.route('/team-builder', methods=['POST'])
def handle_team_builder():
    session['actual_team_list'] = []
    if 'scegli_modulo' in request.form:
        modulo_scelto = request.form.get('modulo_scelto')
        session['actual_modulo_scelto']  = modulo_scelto
        return view_team_builder(player_info, session['actual_modulo_scelto'] )
    
    if 'build_team' in request.form:
        for i in range(1,12):
            session['actual_team_list'].append(request.form.get('nome_'+str(i)))
        return view_builded_team(session['actual_team_list'], player_info, player_hist)


if __name__ == "__main__":
    app.secret_key = 'fantastat'
    app.config['SESSION_TYPE'] = 'filesystem'

    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jul/2022 12:11:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2022 12:11:10] "GET /player-page HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2022 12:11:12] "POST /player-page HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2022 12:11:15] "POST /player-page HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2022 12:11:17] "POST /player-page HTTP/1.1" 200 -
